In [1]:
from dotenv import load_dotenv
import os
import time
import datetime
import json
import requests
from requests.utils import quote
from tqdm.notebook import tqdm
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
load_dotenv('.env')
OMDB_KEY = os.environ.get("OMDB_API_KEY")
omdb = 'http://www.omdbapi.com/?apikey={}&t='.format(OMDB_KEY)

# 1. Netflix

In [ ]:
netflix = pd.read_csv('../data/netflix_titles.csv', sep=',', header=0)
netflix.drop(columns=['show_id', 'date_added', 'description'], inplace=True)
netflix.rename(columns={'type': 'Type', 'title': 'Title', 'director': 'Director', 'cast': 'Actors',
                  'country': 'Country', 'release_year': 'Release', 'rating': 'Rating',
                  'duration': 'Runtime', 'listed_in': 'Genre'}, inplace=True)
netflix = netflix.reindex(columns=['Type', 'Title', 'Director', 'Actors', 'Country', 'Release', 'Rating', 
                                   'Runtime', 'Genre', 'Language', 'IMDb', 'RottenTomatoes', 'Metacritic'])
netflix = netflix[['Title', 'Type', 'Director', 'Actors', 'Release', 'Genre', 'Runtime', 'Language', 
                   'Country', 'Rating', 'IMDb', 'RottenTomatoes', 'Metacritic']]

In [ ]:
netflix.head()

In [ ]:
netflix.info()

In [ ]:
netflix.iloc[1]['Actors']

In [ ]:
listRes = list(netflix.iloc[1]['Actors'].split(", "))
listRes

In [ ]:
def extract_json_data(info, index, dataset):
    if info['Response']=='True':
        #['Title', 'Type', 'Director', 'Actors', 'Release', 'Genre', 'Runtime', 'Language', 
        # 'Country', 'Rating', 'IMDb', 'RottenTomatoes', 'Metacritic']
        row = []
        imdb = 'NaN'
        rotten = 'NaN'
        meta = 'NaN'
        row.append(info['Title'])
        row.append(info['Type'])
        row.append(info['Director'])
        row.append(info['Actors'])
        row.append(info['Released'])
        row.append(info['Genre'])
        if info['Type']=='movie':
            row.append(info['Runtime'])
        else:
            row.append(info['totalSeasons']+' seasons')
        row.append(info['Language'])
        row.append(info['Country'])
        row.append(info['Rated'])
        for rate in info['Ratings']:
            if rate['Source']=='Internet Movie Database':
                imdb = rate['Value']
            elif rate['Source']=='Rotten Tomatoes':
                rotten = rate['Value']
            elif rate['Source']=='Metacritic':
                meta = rate['Value']
        row.append(imdb)
        row.append(rotten)
        row.append(meta)
        dataset.loc[index] = row    

In [ ]:
r = requests.get(omdb+quote(netflix.iloc[1]['Title']))

In [ ]:
jsonRes = r.json()

In [ ]:
jsonRes

In [ ]:
for index, row in tqdm(netflix.iterrows(), total=netflix.shape[0]):
    r = requests.get(omdb+quote(netflix.iloc[index]['Title']))
    extract_json_data(r.json(), index, netflix)

In [ ]:
netflix.info()

In [ ]:
netflix['Type'].unique()

In [ ]:
netflix[netflix['Actors'].isnull()]

In [ ]:
netflix.drop(netflix[netflix.isnull().sum(axis=1)>=6].index, inplace=True)

In [ ]:
netflix.replace('N/A', np.NaN, inplace=True)
netflix.replace('N/A seasons', np.NaN, inplace=True)
netflix.replace('NaN', np.NaN, inplace=True)

In [ ]:
netflix['Actors'] = netflix['Actors'].apply(lambda x: np.NaN if pd.isnull(x) else list(x.split(", ")))
netflix['Genre'] = netflix['Genre'].apply(lambda x: np.NaN if pd.isnull(x) else list(x.split(", ")))
netflix['Language'] = netflix['Language'].apply(lambda x: np.NaN if pd.isnull(x) else list(x.split(", ")))
netflix['Country'] = netflix['Country'].apply(lambda x: np.NaN if pd.isnull(x) else list(x.split(", ")))

In [ ]:
netflix['Actors'] = netflix['Actors'].apply(lambda x: np.NaN if isinstance(x, float) else x[0] if len(x)==1 else x)
netflix['Genre'] = netflix['Genre'].apply(lambda x: np.NaN if isinstance(x, float) else x[0] if len(x)==1 else x)
netflix['Language'] = netflix['Language'].apply(lambda x: np.NaN if isinstance(x, float) else x[0] if len(x)==1 else x)
netflix['Country'] = netflix['Country'].apply(lambda x: np.NaN if isinstance(x, float) else x[0] if len(x)==1 else x)

In [ ]:
netflix['IMDb'] = netflix['IMDb'].apply(lambda x: np.NaN if pd.isnull(x) else x.replace('/10', ''))
netflix['RottenTomatoes'] = netflix['RottenTomatoes'].apply(lambda x: np.NaN if pd.isnull(x) else x.replace('%', ''))
netflix['Metacritic'] = netflix['Metacritic'].apply(lambda x: np.NaN if pd.isnull(x) else x.replace('/100', ''))

In [ ]:
netflix['Runtime'] = netflix['Runtime'].apply(lambda x: np.NaN if pd.isnull(x) else x.replace(' min', ''))
netflix['Runtime'] = netflix['Runtime'].apply(lambda x: np.NaN if pd.isnull(x) else x.replace(' seasons', ''))
netflix['Runtime'] = netflix['Runtime'].apply(lambda x: np.NaN if pd.isnull(x) else x.replace(' Seasons', ''))

In [ ]:
netflix['Type'] = netflix['Type'].apply(lambda x: x.replace('movie', 'Movie'))
netflix['Type'] = netflix['Type'].apply(lambda x: x.replace('series', 'TV Show'))

In [ ]:
netflix.reset_index(drop=True, inplace=True)

In [ ]:
netflix.head()

In [ ]:
netflix.info()

In [ ]:
#netflix.to_csv('../data/netflix.csv')

# 2. Prime Video

In [ ]:
prime = pd.read_csv('../data/amazon_prime_titles.csv', sep=',', header=0)
prime.drop(columns=['show_id', 'date_added', 'description'], inplace=True)
prime.rename(columns={'type': 'Type', 'title': 'Title', 'director': 'Director', 'cast': 'Actors',
                  'country': 'Country', 'release_year': 'Release', 'rating': 'Rating',
                  'duration': 'Runtime', 'listed_in': 'Genre'}, inplace=True)
prime = prime.reindex(columns=['Type', 'Title', 'Director', 'Actors', 'Country', 'Release', 'Rating', 
                                   'Runtime', 'Genre', 'Language', 'IMDb', 'RottenTomatoes', 'Metacritic'])
prime = prime[['Title', 'Type', 'Director', 'Actors', 'Release', 'Genre', 'Runtime', 'Language', 
                   'Country', 'Rating', 'IMDb', 'RottenTomatoes', 'Metacritic']]

In [ ]:
prime.head()

In [ ]:
prime.info()

In [ ]:
for index, row in tqdm(prime.iterrows(), total=prime.shape[0]):
    r = requests.get(omdb+quote(prime.iloc[index]['Title']))
    extract_json_data(r.json(), index, prime)

In [ ]:
prime.info()

In [ ]:
prime.head()

In [ ]:
#prime.to_csv('../data/prime.csv')

In [ ]:
prime['Type'].unique()

In [ ]:
prime[prime['Actors'].isnull()]

In [ ]:
prime[prime.isnull().sum(axis=1)>=7]

In [ ]:
prime.replace('N/A', np.NaN, inplace=True)
prime.replace('N/A seasons', np.NaN, inplace=True)
prime.replace('NaN', np.NaN, inplace=True)

In [ ]:
prime.drop(prime[prime.isnull().sum(axis=1)>=6].index, inplace=True)

In [ ]:
prime.info()

In [ ]:
prime['Actors'] = prime['Actors'].apply(lambda x: np.NaN if pd.isnull(x) else list(x.split(", ")))
prime['Genre'] = prime['Genre'].apply(lambda x: np.NaN if pd.isnull(x) else list(x.split(", ")))
prime['Language'] = prime['Language'].apply(lambda x: np.NaN if pd.isnull(x) else list(x.split(", ")))
prime['Country'] = prime['Country'].apply(lambda x: np.NaN if pd.isnull(x) else list(x.split(", ")))

In [ ]:
prime['Actors'] = prime['Actors'].apply(lambda x: np.NaN if isinstance(x, float) else x[0] if len(x)==1 else x)
prime['Genre'] = prime['Genre'].apply(lambda x: np.NaN if isinstance(x, float) else x[0] if len(x)==1 else x)
prime['Language'] = prime['Language'].apply(lambda x: np.NaN if isinstance(x, float) else x[0] if len(x)==1 else x)
prime['Country'] = prime['Country'].apply(lambda x: np.NaN if isinstance(x, float) else x[0] if len(x)==1 else x)

In [ ]:
prime['IMDb'] = prime['IMDb'].apply(lambda x: np.NaN if pd.isnull(x) else x.replace('/10', ''))
prime['RottenTomatoes'] = prime['RottenTomatoes'].apply(lambda x: np.NaN if pd.isnull(x) else x.replace('%', ''))
prime['Metacritic'] = prime['Metacritic'].apply(lambda x: np.NaN if pd.isnull(x) else x.replace('/100', ''))

In [ ]:
prime['Runtime'] = prime['Runtime'].apply(lambda x: np.NaN if pd.isnull(x) else x.replace(' min', ''))
prime['Runtime'] = prime['Runtime'].apply(lambda x: np.NaN if pd.isnull(x) else x.replace(' seasons', ''))
prime['Runtime'] = prime['Runtime'].apply(lambda x: np.NaN if pd.isnull(x) else x.replace(' Seasons', ''))

In [ ]:
prime['Type'] = prime['Type'].apply(lambda x: x.replace('movie', 'Movie'))
prime['Type'] = prime['Type'].apply(lambda x: x.replace('series', 'TV Show'))

In [ ]:
prime.reset_index(drop=True, inplace=True)

In [ ]:
prime.head()

In [ ]:
prime.info()

In [ ]:
#prime.to_csv('../data/prime.csv')

# 3. Disney+

In [ ]:
disney = pd.read_csv('../data/disney_plus_titles.csv', sep=',', header=0)
disney.drop(columns=['show_id', 'date_added', 'description'], inplace=True)
disney.rename(columns={'type': 'Type', 'title': 'Title', 'director': 'Director', 'cast': 'Actors',
                  'country': 'Country', 'release_year': 'Release', 'rating': 'Rating',
                  'duration': 'Runtime', 'listed_in': 'Genre'}, inplace=True)
disney = disney.reindex(columns=['Type', 'Title', 'Director', 'Actors', 'Country', 'Release', 'Rating', 
                                   'Runtime', 'Genre', 'Language', 'IMDb', 'RottenTomatoes', 'Metacritic'])
disney = disney[['Title', 'Type', 'Director', 'Actors', 'Release', 'Genre', 'Runtime', 'Language', 
                   'Country', 'Rating', 'IMDb', 'RottenTomatoes', 'Metacritic']]

In [ ]:
disney.head()

In [ ]:
disney.info()

In [ ]:
for index, row in tqdm(disney.iterrows(), total=disney.shape[0]):
    r = requests.get(omdb+quote(disney.iloc[index]['Title']))
    extract_json_data(r.json(), index, disney)

In [ ]:
disney.info()

In [ ]:
disney.head()

In [ ]:
#disney.to_csv('../data/disney.csv')

In [ ]:
disney['Type'].unique()

In [ ]:
disney[disney['Actors'].isnull()]

In [ ]:
disney[disney.isnull().sum(axis=1)>=6]

In [ ]:
disney.replace('N/A', np.NaN, inplace=True)
disney.replace('N/A seasons', np.NaN, inplace=True)
disney.replace('NaN', np.NaN, inplace=True)

In [ ]:
disney.drop(disney[disney.isnull().sum(axis=1)>=6].index, inplace=True)

In [ ]:
disney.info()

In [ ]:
disney['Actors'] = disney['Actors'].apply(lambda x: np.NaN if pd.isnull(x) else list(x.split(", ")))
disney['Genre'] = disney['Genre'].apply(lambda x: np.NaN if pd.isnull(x) else list(x.split(", ")))
disney['Language'] = disney['Language'].apply(lambda x: np.NaN if pd.isnull(x) else list(x.split(", ")))
disney['Country'] = disney['Country'].apply(lambda x: np.NaN if pd.isnull(x) else list(x.split(", ")))

In [ ]:
disney['Actors'] = disney['Actors'].apply(lambda x: np.NaN if isinstance(x, float) else x[0] if len(x)==1 else x)
disney['Genre'] = disney['Genre'].apply(lambda x: np.NaN if isinstance(x, float) else x[0] if len(x)==1 else x)
disney['Language'] = disney['Language'].apply(lambda x: np.NaN if isinstance(x, float) else x[0] if len(x)==1 else x)
disney['Country'] = disney['Country'].apply(lambda x: np.NaN if isinstance(x, float) else x[0] if len(x)==1 else x)

In [ ]:
disney['IMDb'] = disney['IMDb'].apply(lambda x: np.NaN if pd.isnull(x) else x.replace('/10', ''))
disney['RottenTomatoes'] = disney['RottenTomatoes'].apply(lambda x: np.NaN if pd.isnull(x) else x.replace('%', ''))
disney['Metacritic'] = disney['Metacritic'].apply(lambda x: np.NaN if pd.isnull(x) else x.replace('/100', ''))

In [ ]:
disney['Runtime'] = disney['Runtime'].apply(lambda x: np.NaN if pd.isnull(x) else x.replace(' min', ''))
disney['Runtime'] = disney['Runtime'].apply(lambda x: np.NaN if pd.isnull(x) else x.replace(' seasons', ''))
disney['Runtime'] = disney['Runtime'].apply(lambda x: np.NaN if pd.isnull(x) else x.replace(' Seasons', ''))

In [ ]:
disney['Type'] = disney['Type'].apply(lambda x: x.replace('movie', 'Movie'))
disney['Type'] = disney['Type'].apply(lambda x: x.replace('series', 'TV Show'))

In [ ]:
disney.reset_index(drop=True, inplace=True)

In [ ]:
disney.head()

In [ ]:
disney.info()

In [ ]:
#disney.to_csv('../data/disney.csv')

# Merge datasets

In [15]:
netflix = pd.read_csv('../data/netflix.csv', sep=',', header=0, index_col=0)
netflix['Platform'] = 'Netflix'
prime = pd.read_csv('../data/prime.csv', sep=',', header=0, index_col=0)
prime['Platform'] = 'Prime'
disney = pd.read_csv('../data/disney.csv', sep=',', header=0, index_col=0)
disney['Platform'] = 'Disney'

In [18]:
netflix.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8453 entries, 0 to 8452
Data columns (total 14 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Title           8453 non-null   object 
 1   Type            8453 non-null   object 
 2   Director        6241 non-null   object 
 3   Actors          8284 non-null   object 
 4   Release         8275 non-null   object 
 5   Genre           8446 non-null   object 
 6   Runtime         8295 non-null   object 
 7   Language        7494 non-null   object 
 8   Country         8249 non-null   object 
 9   Rating          6136 non-null   object 
 10  IMDb            7520 non-null   float64
 11  RottenTomatoes  2846 non-null   float64
 12  Metacritic      2079 non-null   float64
 13  Platform        8453 non-null   object 
dtypes: float64(3), object(11)
memory usage: 990.6+ KB


In [19]:
prime.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8271 entries, 0 to 8270
Data columns (total 14 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Title           8271 non-null   object 
 1   Type            8271 non-null   object 
 2   Director        7149 non-null   object 
 3   Actors          8164 non-null   object 
 4   Release         7970 non-null   object 
 5   Genre           8243 non-null   object 
 6   Runtime         8021 non-null   object 
 7   Language        6912 non-null   object 
 8   Country         7043 non-null   object 
 9   Rating          5765 non-null   object 
 10  IMDb            6662 non-null   float64
 11  RottenTomatoes  2086 non-null   float64
 12  Metacritic      1343 non-null   float64
 13  Platform        8271 non-null   object 
dtypes: float64(3), object(11)
memory usage: 969.3+ KB


In [20]:
disney.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1328 entries, 0 to 1327
Data columns (total 14 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Title           1328 non-null   object 
 1   Type            1328 non-null   object 
 2   Director        994 non-null    object 
 3   Actors          1300 non-null   object 
 4   Release         1322 non-null   object 
 5   Genre           1327 non-null   object 
 6   Runtime         1307 non-null   object 
 7   Language        1104 non-null   object 
 8   Country         1297 non-null   object 
 9   Rating          1173 non-null   object 
 10  IMDb            1094 non-null   float64
 11  RottenTomatoes  443 non-null    float64
 12  Metacritic      370 non-null    float64
 13  Platform        1328 non-null   object 
dtypes: float64(3), object(11)
memory usage: 155.6+ KB


In [24]:
df = pd.concat([netflix, prime, disney], ignore_index=True)

In [25]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18052 entries, 0 to 18051
Data columns (total 14 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Title           18052 non-null  object 
 1   Type            18052 non-null  object 
 2   Director        14384 non-null  object 
 3   Actors          17748 non-null  object 
 4   Release         17567 non-null  object 
 5   Genre           18016 non-null  object 
 6   Runtime         17623 non-null  object 
 7   Language        15510 non-null  object 
 8   Country         16589 non-null  object 
 9   Rating          13074 non-null  object 
 10  IMDb            15276 non-null  float64
 11  RottenTomatoes  5375 non-null   float64
 12  Metacritic      3792 non-null   float64
 13  Platform        18052 non-null  object 
dtypes: float64(3), object(11)
memory usage: 1.9+ MB


In [26]:
df.head()

,Title,Type,Director,Actors,Release,Genre,Runtime,Language,Country,Rating,IMDb,RottenTomatoes,Metacritic,Platform
0,Dick Johnson Is Dead,Movie,Kirsten Johnson,"['Michael Hilow', 'Ana Hoffman', 'Dick Johnson']",02 Oct 2020,"['Documentary', 'Drama']",89,English,United States,PG-13,7.4,100.0,89.0,Netflix
1,Blood & Water,TV Show,NaN,"['Ama Qamata', 'Khosi Ngema', 'Gail Mabalane']",20 May 2020,"['Drama', 'Mystery']",2,English,South Africa,TV-MA,6.6,NaN,NaN,Netflix
2,Ganglands,TV Show,NaN,"['Nabiha Akkari', 'Sami Bouajila', 'Salim Kech...",24 Sep 2021,"['Action', 'Crime', 'Drama']",1,French,France,TV-MA,7.0,NaN,NaN,Netflix
3,Jailbirds New Orleans,TV Show,NaN,NaN,24 Sep 2021,"['Documentary', 'Reality-TV']",1,English,United States,TV-MA,6.7,NaN,NaN,Netflix
4,Kota Factory,TV Show,NaN,"['Mayur More', 'Jitendra Kumar', 'Ranjan Raj']",16 Apr 2019,"['Comedy', 'Drama']",2,Hindi,India,TV-14,9.2,NaN,NaN,Netflix


In [27]:
df.to_csv('../data/data.csv')

In [108]:
netflix_rev = pd.read_csv('../data/netflix_revenues.csv', sep=',', header=0, index_col=0)
netflix_rev['Platform'] = 'Netflix'
prime_rev = pd.read_csv('../data/prime_revenues.csv', sep=',', header=0, index_col=0)
prime_rev['Platform'] = 'Prime'
disney_rev = pd.read_csv('../data/disney_revenues.csv', sep=',', header=0, index_col=0)
disney_rev['Platform'] = 'Disney'

In [109]:
netflix_rev['Revenue'] = netflix_rev.groupby(['Year', 'Quarter'])[['Revenue']].transform('sum')
netflix_rev['Subscriptions'] = netflix_rev.groupby(['Year', 'Quarter'])[['Subscriptions']].transform('sum')
netflix_rev.drop('Area', axis=1, inplace=True)
netflix_rev.drop_duplicates(inplace=True, ignore_index=True)

In [111]:
netflix_rev.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16 entries, 0 to 15
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Year           16 non-null     int64 
 1   Quarter        16 non-null     object
 2   Revenue        16 non-null     int64 
 3   Subscriptions  16 non-null     int64 
 4   Platform       16 non-null     object
dtypes: int64(3), object(2)
memory usage: 768.0+ bytes


In [112]:
prime_rev.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 16 entries, 0 to 15
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Year           16 non-null     int64 
 1   Quarter        16 non-null     object
 2   Revenue        16 non-null     int64 
 3   Subscriptions  16 non-null     int64 
 4   Platform       16 non-null     object
dtypes: int64(3), object(2)
memory usage: 768.0+ bytes


In [113]:
disney_rev.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8 entries, 0 to 7
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Year           8 non-null      int64 
 1   Quarter        8 non-null      object
 2   Revenue        8 non-null      int64 
 3   Subscriptions  8 non-null      int64 
 4   Platform       8 non-null      object
dtypes: int64(3), object(2)
memory usage: 384.0+ bytes


In [114]:
df_rev = pd.concat([netflix_rev, prime_rev, disney_rev], ignore_index=True)

In [115]:
df_rev.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40 entries, 0 to 39
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Year           40 non-null     int64 
 1   Quarter        40 non-null     object
 2   Revenue        40 non-null     int64 
 3   Subscriptions  40 non-null     int64 
 4   Platform       40 non-null     object
dtypes: int64(3), object(2)
memory usage: 1.7+ KB


In [116]:
df_rev.head()

,Year,Quarter,Revenue,Subscriptions,Platform
0,2018,Q1,3602000000,118910000,Netflix
1,2018,Q2,3814000000,124350000,Netflix
2,2018,Q3,3910000000,130420000,Netflix
3,2018,Q4,4102000000,139260000,Netflix
4,2019,Q1,4440000000,148860000,Netflix


In [117]:
df_rev.to_csv('../data/revenues.csv')